# Imputación de Datos con KNNImputer

In [1]:
import pandas as pd
from sklearn.impute import KNNImputer

Lectura del dataset

In [2]:
df = pd.read_csv('../data/processed/dataset_municipal_v2.csv')
df.head()

,cve_mun,municipio,pob_total,pob_indigena,pob_afrodescendiente,viv,viv_internet,viv_computadoras,viv_celular,ind_rez_social,...,licenciatura_percentage,especialidad_percentage,maestr_a_percentage,doctorado_percentage,indice_carencia_compuesto,pct_hacinamiento,pct_agua_entubada,pct_drenaje,pct_piso_tierra,pct_uso_lena
0,20001,Abejones,841,815,1,217,17,17,92,0.815821,...,12.646612,0.173986,0.79035,0.116028,19.447005,3.23,0.46,23.04,70.51,0.0
1,20002,Acatlán de Pérez Figueroa,45167,8548,316,12859,3700,1715,10141,0.066739,...,12.646612,0.173986,0.79035,0.116028,19.709153,8.72,5.52,5.44,78.87,0.0
2,20003,Asunción Cacalotepec,2547,2504,0,794,43,54,202,1.873466,...,12.646612,0.173986,0.79035,0.116028,31.435768,3.65,8.31,75.06,70.15,0.0
3,20004,Asunción Cuyotepeji,1107,3,15,342,60,47,118,-0.158009,...,12.646612,0.173986,0.79035,0.116028,19.532164,5.85,1.46,10.23,80.12,0.0
4,20005,Asunción Ixtaltepec,15261,10406,723,4828,1409,1013,3733,-0.494904,...,12.646612,0.173986,0.79035,0.116028,16.872411,6.30,1.82,2.55,73.70,0.0


Columnas con datos faltantes

In [3]:
to_keep = ["gini", "tasa_ingresos", "total_elementos_salud"]
all_cols_missing = []

# show count of missing values per column
for col in df.columns:
    missing_count = df[col].isnull().sum()
    if missing_count > 0:
        print(f"{col}: {missing_count} missing values")
        all_cols_missing.append(col)

# remove columns with missing values except those in to_keep
to_remove = [col for col in all_cols_missing if col not in to_keep]
print(f"\nRemoving ({len(to_remove)}) columns: {to_remove}")
df_cleaned = df.drop(columns=to_remove)

gini: 4 missing values
tasa_ingresos: 10 missing values
cama_hospital: 534 missing values
cama_fuera_hospital: 475 missing values
cama_rec_nac: 537 missing values
consultorios: 167 missing values
equipamento: 510 missing values
medicos_otros: 531 missing values
medicos_gen_esp_odont: 199 missing values
otro_pers: 455 missing values
otro_pers_prof: 446 missing values
enfermeria_contac_paciente: 180 missing values
enfermeria_otras: 530 missing values
medicos_adiest: 319 missing values
per_tecnico: 421 missing values
total_elementos_salud: 167 missing values

Removing (13) columns: ['cama_hospital', 'cama_fuera_hospital', 'cama_rec_nac', 'consultorios', 'equipamento', 'medicos_otros', 'medicos_gen_esp_odont', 'otro_pers', 'otro_pers_prof', 'enfermeria_contac_paciente', 'enfermeria_otras', 'medicos_adiest', 'per_tecnico']


Ejemplos de valores faltantes para las columnas seleccionadas

In [5]:
# for each column in to_keep, show the first 3 rows with missing values
to_show = []
for col in to_keep:
    missing_rows = df_cleaned[df_cleaned[col].isnull()].head(3)
    if not missing_rows.empty:
        print(f"\nPrimeras 3 instancias con valores vacíos en '{col}':")
        print(missing_rows[["municipio", col]])
        to_show.append({
            "column": col,
            "rows": missing_rows.index
        })


Primeras 3 instancias con valores vacíos en 'gini':
                  municipio  gini
56   Matías Romero Avendaño   NaN
139   San Francisco Chindúa   NaN
406   Santa María Chimalapa   NaN

Primeras 3 instancias con valores vacíos en 'tasa_ingresos':
                  municipio  tasa_ingresos
47       Magdalena Mixtepec            NaN
98     San Andrés Tepetlapa            NaN
145  San Francisco Logueche            NaN

Primeras 3 instancias con valores vacíos en 'total_elementos_salud':
                 municipio  total_elementos_salud
3      Asunción Cuyotepeji                    NaN
10               Calihualá                    NaN
19  Constancia del Rosario                    NaN


Imputación de los datos

In [6]:
# imput with KNNImputer
sub_df = df_cleaned[to_keep]
imputer = KNNImputer(n_neighbors=19)
imputed = imputer.fit_transform(sub_df)

# Re-add to df_cleaned
for i, col in enumerate(to_keep):
    df_cleaned[col] = imputed[:, i]

# Ceil round for 'total_elementos_salud'
df_cleaned['total_elementos_salud'] = df_cleaned['total_elementos_salud'].round().astype(int)

Mostrar los valores imputados

In [7]:
for item in to_show:
    col = item["column"]
    rows = item["rows"]
    print(f"\nValores imputados para '{col}':")
    print(df_cleaned.loc[rows, ["municipio", col]])


Valores imputados para 'gini':
                  municipio      gini
56   Matías Romero Avendaño  0.395994
139   San Francisco Chindúa  0.374153
406   Santa María Chimalapa  0.376627

Valores imputados para 'tasa_ingresos':
                  municipio  tasa_ingresos
47       Magdalena Mixtepec       0.189970
98     San Andrés Tepetlapa       0.176740
145  San Francisco Logueche       0.214754

Valores imputados para 'total_elementos_salud':
                 municipio  total_elementos_salud
3      Asunción Cuyotepeji                     16
10               Calihualá                     12
19  Constancia del Rosario                     39
